### Classification of images with Convolutional Neural Network
Goal is to classify a given image into either a dog or a cat.

#### Data pre-processing

##### Libraries to process data

In [3]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

##### Processing training data
To avoid overfitting, same images are transformed (augmented) by rotating, zooming, flipping etc

##### Create image transformer

In [7]:
train_data_transformer = ImageDataGenerator(rescale = 1./255 ,    # for scaling the image
                                   shear_range = 0.2,            # for distorting/shearing the image , like pulling diagonally
                                   zoom_range = 0.2,             # zooming the image
                                   horizontal_flip = True)       # for flipping


##### Transforming training set
Using flow_from_directory() , passing the training images to the image transformer

In [8]:
training_set = train_data_transformer.flow_from_directory(r'dataset\training_set',
                                                 target_size = (64, 64), # final image size
                                                 batch_size = 32,        # no. of images in a batch
                                                 class_mode = 'binary')  # type of outcome, here, it's cat or dog, so it si binary

Found 8000 images belonging to 2 classes.


##### Processing test data
Can keep the test data intact but it needs to be on the same scale as the training data. So transforming with only rescale argument.

In [9]:
test_data_transformer = ImageDataGenerator(rescale = 1./255)

##### Transforming test set
Using flow_from_directory() , passing the test images to the image transformer

In [10]:
test_set = test_data_transformer.flow_from_directory(r'dataset\test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


#### Building Convolutional Neural Network

##### Instantiating a neural network with sequential class

In [12]:
cnn = tf.keras.models.Sequential()

##### Adding Convolution layer
Using 32 filters with a kernel(feature detector) size of 3x3 and 'relu' activation and specifying the input shape (same as the training_set output)

Value 3 in input_shape=[64, 64, 3] indicates that the input in a coloured image set

In [13]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

##### Adding pooling layer
Using pool_size of 2 which is the frame size of pooling frame and strides of 2 indicating that the pooling frame strides 2 pixels

In [14]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

##### Adding another Convolution layer

In [16]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))

##### Adding another pooling layer

In [17]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

##### Adding Flattening layer
To flatten the results of convolution and pooling layers into one dimensional vector to be used as input to fully connected neural network layers

In [18]:
cnn.add(tf.keras.layers.Flatten())

##### Adding fully connected neural network layer
Using a high value of 128 for units (neurons) as image classification is more complex and a higher number of neurons improve the accuracy

In [20]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

##### Adding the output layer
As output is bunary, using units = 1 and using sigmoid activation to understand probability. Softmax is used for multi class classification.

In [21]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

#### Training the Convolutional Neural Network
'adam' optimizer and 'binary_crossentropy' is used as it is binary classification

In [22]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

#### Training and Evaluation
This trains the CNN on training dataset and evaluates with test dataset

In [23]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
250/250 [==============================] - 37s 124ms/step - loss: 0.6783 - accuracy: 0.5710 - val_loss: 0.6542 - val_accuracy: 0.6215
Epoch 2/25
250/250 [==============================] - 25s 100ms/step - loss: 0.6271 - accuracy: 0.6484 - val_loss: 0.6536 - val_accuracy: 0.6310
Epoch 3/25
250/250 [==============================] - 25s 99ms/step - loss: 0.5929 - accuracy: 0.6924 - val_loss: 0.5975 - val_accuracy: 0.6945
Epoch 4/25
250/250 [==============================] - 25s 100ms/step - loss: 0.5638 - accuracy: 0.7129 - val_loss: 0.5497 - val_accuracy: 0.7275
Epoch 5/25
250/250 [==============================] - 25s 100ms/step - loss: 0.5425 - accuracy: 0.7287 - val_loss: 0.5326 - val_accuracy: 0.7485
Epoch 6/25
250/250 [==============================] - 25s 101ms/step - loss: 0.5098 - accuracy: 0.7500 - val_loss: 0.5806 - val_accuracy: 0.7045
Epoch 7/25
250/250 [==============================] - 26s 103ms/step - loss: 0.4759 - accuracy: 0.7751 - val_loss: 0.5047 - val_acc

#### Prediction on a random new image

##### Loading new image for prediction
Here a random dog image from internet is used for prediction after changing the target size to the same size as training image size

In [27]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img(r'random_dog_image.jpeg', target_size = (64, 64))

##### Converting the image to array

In [28]:
test_image = image.img_to_array(test_image)

##### Adding extra dimension 
As training was done in batches of 32 and here in prediction we need to add an additional dummy dimension in place of batch.

In [29]:
test_image = np.expand_dims(test_image, axis = 0)

#### Predicting and showing result

In [30]:
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'
print(prediction)

dog


##### Repeating prediction for a randon cat image

In [32]:
test_image = image.load_img(r'kitten.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'
print(prediction)

cat
